In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Carregar os dados
data = pd.read_csv('all_stocks_5yr.csv')
data.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [ ]:
train_open = data.loc[:, ['open']].values
train_open

array([[15.07],
       [14.89],
       [14.45],
       ...,
       [76.64],
       [72.74],
       [72.7 ]])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
train_scaled = scaler.fit_transform(train_open)
train_scaled

array([[0.00658545],
       [0.00649732],
       [0.00628189],
       ...,
       [0.03673166],
       [0.03482212],
       [0.03480253]])

In [ ]:
x_train = []
y_train = []
timesteps = 50

for i in range(timesteps, 1250):
  x_train.append(train_scaled[i - timesteps: i, 0])
  y_train.append(train_scaled[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Modelo RNN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout

In [135]:
regressor = Sequential()

#Adding the first RNN layer and some Dropout regularization
regressor.add(SimpleRNN(units = 50, activation='tanh', return_sequences=True, input_shape= (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

#Adding the second RNN layer and some Dropout regularization
regressor.add(SimpleRNN(units = 50, activation='tanh', return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 50, activation='tanh', return_sequences=True))
regressor.add(Dropout(0.2))

#Adding the fourth RNN layer and some Dropout regularization
regressor.add(SimpleRNN(units = 50))
regressor.add(Dropout(0.2))

#Adding the output layer
regressor.add(Dense(units = 1))

#Compile the RNN
regressor.compile(optimizer='adam', loss='mean_squared_error', metrics = ["accuracy"])

#Fitting the RNN to the Training set
x=regressor.fit(x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
38/38 [==============================] - 7s 77ms/step - loss: 0.3161 - accuracy: 8.3333e-04
Epoch 2/100
38/38 [==============================] - 4s 97ms/step - loss: 0.2165 - accuracy: 8.3333e-04
Epoch 3/100
38/38 [==============================] - 3s 68ms/step - loss: 0.1355 - accuracy: 0.0017
Epoch 4/100
38/38 [==============================] - 3s 68ms/step - loss: 0.0842 - accuracy: 0.0017
Epoch 5/100
38/38 [==============================] - 4s 96ms/step - loss: 0.0468 - accuracy: 0.0017
Epoch 6/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0358 - accuracy: 8.3333e-04
Epoch 7/100
38/38 [==============================] - 2s 52ms/step - loss: 0.0293 - accuracy: 0.0017
Epoch 8/100
38/38 [==============================] - 2s 49ms/step - loss: 0.0229 - accuracy: 0.0017
Epoch 9/100
38/38 [==============================] - 3s 74ms/step - loss: 0.0192 - accuracy: 0.0017
Epoch 10/100
38/38 [==============================] - 4s 95ms/step - loss: 0.0149 - accu

# Resultados do treinamento

In [ ]:
x.history["loss"]

[0.21347007155418396,
 0.05679818615317345,
 0.010309458710253239,
 0.004259772133082151,
 0.002188229700550437,
 0.0016152334865182638,
 0.0012228854466229677,
 0.0009510427480563521,
 0.0008319935295730829,
 0.0007291224901564419,
 0.000623780011665076,
 0.0005886288709007204,
 0.000506202457472682,
 0.0004947050474584103,
 0.00045220027095638216,
 0.0004577889631036669,
 0.0004708382475655526,
 0.0004005588998552412,
 0.0003691788879223168,
 0.0003438702260609716,
 0.00032390255364589393,
 0.00028607287094928324,
 0.00027180713368579745,
 0.00025736470706760883,
 0.0002456768706906587,
 0.00024874924565665424,
 0.0002261093322886154,
 0.00022496226301882416,
 0.0002075731026707217,
 0.00018210340931545943,
 0.00018340065435040742,
 0.00020588704501278698,
 0.00017649422807153314,
 0.00017682624456938356,
 0.00016689178301021457,
 0.00015026744222268462,
 0.0001672593061812222,
 0.00016584689728915691,
 0.00015091840759851038,
 0.00014413097233045846,
 0.00013034199946559966,
 0.0001

# Modelo de Predição RNN

In [ ]:
y_predictions = regressor.predict(x_train)
y_predictions = scaler.inverse_transform(y_predictions)
y_predictions.shape

38/38 [==============================] - 1s 13ms/step


(1200, 1)

In [ ]:
y_train = np.reshape(y_train, (y_train.shape[0],1))
y_train = scaler.inverse_transform(y_train)
y_train.shape

(1200, 1)

# Testes de Validação

In [ ]:
length_data = len(data)
split_ratio = 0.7
length_train = round(length_data * split_ratio)
length_validation = length_data - length_train

In [ ]:
valid_data = data[length_train:].iloc[:,:2]
valid_data['date'] = pd.to_datetime(valid_data['date'])
valid_data

,date,open
433328,2016-04-21,75.21
433329,2016-04-22,74.92
433330,2016-04-25,75.26
433331,2016-04-26,75.75
433332,2016-04-27,76.76
...,...,...
619035,2018-02-01,76.84
619036,2018-02-02,77.53
619037,2018-02-05,76.64
619038,2018-02-06,72.74


In [ ]:
dataset_validation = valid_data.values
dataset_validation = np.reshape(dataset_validation, (-1,1))
scaled_dataset_validation =  scaler.fit_transform(train_open)
print("Shape of scaled validation dataset :",scaled_dataset_validation.shape)

Shape of scaled validation dataset : (619040, 1)


In [ ]:
X_test = []
y_test = []

for i in range(timesteps, length_validation):
    X_test.append(scaled_dataset_validation[i-timesteps:i,0])
    y_test.append(scaled_dataset_validation[i,0])


In [ ]:
X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
y_test = np.reshape(y_test, (-1,1))

In [ ]:
print("Shape of X_test after reshape :",X_test.shape)
print("Shape of y_test after reshape :",y_test.shape)

Shape of X_test after reshape : (185662, 50, 1)
Shape of y_test after reshape : (185662, 1)


In [ ]:
y_pred_of_test = regressor.predict(y_test)
y_pred_of_test = scaler.inverse_transform(y_pred_of_test)
print("Shape of y_pred_of_test :",y_pred_of_test.shape)

5802/5802 [==============================] - 15s 2ms/step
Shape of y_pred_of_test : (185662, 1)


# Modelo LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
testX, testy = np.array(x_train), np.array(y_train)
trainX, trainy = np.array(x_train), np.array(y_train)


In [ ]:
trainX.shape

(1200, 50)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0],1,  trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1,  testX.shape[1]))

In [ ]:
trainy.shape


(1200, 1)

In [ ]:
testX.shape

(1200, 1, 50)

In [134]:
model_lstm = Sequential()
model_lstm.add(
    LSTM(64,return_sequences=True,input_shape = (X_train.shape[1],1))) #64 lstm neuron block
model_lstm.add(
    LSTM(64, return_sequences= False))
model_lstm.add(Dense(32))
model_lstm.add(Dense(1))
model_lstm.compile(loss = "mean_squared_error", optimizer = "adam", metrics = ["accuracy"])
history2 = model_lstm.fit(X_train, y_train, epochs = 100, batch_size = 10)

Epoch 1/100
120/120 [==============================] - 8s 34ms/step - loss: 0.0685 - accuracy: 0.0017
Epoch 2/100
120/120 [==============================] - 4s 37ms/step - loss: 0.0237 - accuracy: 0.0017
Epoch 3/100
120/120 [==============================] - 5s 41ms/step - loss: 0.0100 - accuracy: 0.0017
Epoch 4/100
120/120 [==============================] - 4s 35ms/step - loss: 0.0081 - accuracy: 0.0017
Epoch 5/100
120/120 [==============================] - 7s 55ms/step - loss: 0.0068 - accuracy: 0.0017
Epoch 6/100
120/120 [==============================] - 5s 39ms/step - loss: 0.0043 - accuracy: 0.0017
Epoch 7/100
120/120 [==============================] - 4s 34ms/step - loss: 0.0039 - accuracy: 0.0017
Epoch 8/100
120/120 [==============================] - 5s 41ms/step - loss: 0.0043 - accuracy: 0.0017
Epoch 9/100
120/120 [==============================] - 5s 40ms/step - loss: 0.0031 - accuracy: 0.0017
Epoch 10/100
120/120 [==============================] - 4s 32ms/step - loss: 0.002

#Predição

In [ ]:
y_train = scaler.fit_transform(y_train)

In [136]:
trainPredict = model_lstm.predict(X_train)
testPredict = model_lstm.predict(testX)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainy = scaler.inverse_transform([trainy])
testPredict = scaler.inverse_transform(testPredict)
testy = scaler.inverse_transform([testy])

import math
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainy[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testy[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

38/38 [==============================] - 2s 19ms/step


InvalidArgumentError: Graph execution error:

Detected at node while/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-

# GRU


In [138]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(32,return_sequences=True,input_shape=(10,1)))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dropout(0.20))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam',  metrics = ["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 10, 32)            3360      
                                                                 
 gru_1 (GRU)                 (None, 10, 32)            6336      
                                                                 
 gru_2 (GRU)                 (None, 32)                6336      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 16065 (62.75 KB)
Trainable params: 16065 (62.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1200, 50, 1)
X_test:  (185662, 50, 1)


In [ ]:
y_train.shape

(1200, 1)

In [ ]:
history = model.fit(X_train,testX,validation_data=(y_train,testy), epochs=100,batch_size=32,verbose=1)

Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

NameError: name 'train_predict' is not defined

--------------------------------------------------------------------------------------

In [ ]:
y_train = scaler.fit_transform(y_train)

In [ ]:


model_lstm = Sequential()
model_lstm.add(
    LSTM(64,return_sequences=True,input_shape = (X_train.shape[1],1))) #64 lstm neuron block
model_lstm.add(
    LSTM(64, return_sequences= False))
model_lstm.add(Dense(32))
model_lstm.add(Dense(1))
model_lstm.compile(loss = "mean_squared_error", optimizer = "adam", metrics = ["accuracy"])
history2 = model_lstm.fit(X_train, y_train, epochs = 10, batch_size = 10)

Epoch 1/10
120/120 [==============================] - 9s 48ms/step - loss: 0.0548 - accuracy: 8.3333e-04
Epoch 2/10
120/120 [==============================] - 4s 34ms/step - loss: 0.0086 - accuracy: 0.0017
Epoch 3/10
120/120 [==============================] - 4s 35ms/step - loss: 0.0056 - accuracy: 0.0017
Epoch 4/10
120/120 [==============================] - 6s 46ms/step - loss: 0.0047 - accuracy: 0.0017
Epoch 5/10
120/120 [==============================] - 4s 35ms/step - loss: 0.0042 - accuracy: 0.0017
Epoch 6/10
120/120 [==============================] - 4s 34ms/step - loss: 0.0037 - accuracy: 0.0017
Epoch 7/10
120/120 [==============================] - 6s 46ms/step - loss: 0.0032 - accuracy: 0.0017
Epoch 8/10
120/120 [==============================] - 4s 32ms/step - loss: 0.0034 - accuracy: 0.0017
Epoch 9/10
120/120 [==============================] - 4s 34ms/step - loss: 0.0032 - accuracy: 0.0017
Epoch 10/10
120/120 [==============================] - 6s 46ms/step - loss: 0.0033 - ac

In [ ]:
data.iloc[-1]

date      2013-03-18
open            43.0
high           43.54
low            42.75
close            NaN
volume           NaN
Name             NaN
Name: 65341, dtype: object

In [ ]:
X_input = data.iloc[-timesteps:].values
#X_input = np.reshape(X_input, (-1,1))
#X_input = scaler.fit_transform(X_input)
X_input = np.reshape(X_input, (1,50,1))
print("Shape of X_input :", X_input.shape)
X_input

ValueError: cannot reshape array of size 350 into shape (1,50,1)

In [ ]:
simple_RNN_prediction = scaler.inverse_transform(regressor.predict(X_train))
LSTM_prediction = scaler.inverse_transform(model_lstm.predict(X_train))
print("Simple RNN, Open price prediction for 3/18/2017      :", simple_RNN_prediction[0,0])
print("LSTM prediction, Open price prediction for 3/18/2017 :", LSTM_prediction[0,0])

TypeError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/base_rnn.py", line 662, in call
        timesteps = input_shape[0] if self.time_major else in